In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date
from components.tickers import *

start_date = date.fromisoformat("2015-01-03")

In [2]:
engine = sqlalchemy.create_engine('sqlite:///database.db')

In [3]:
ticker1 = yf.Tickers(tickers_2_20)

dataframe1 = ticker1.history(
    start=start_date, interval="1d", rounding=True, auto_adjust=True).Close


[*********************100%***********************]  226 of 226 completed


In [4]:
dataframe1 = dataframe1.reset_index()

In [5]:
dataframe1['Date'] = pd.DatetimeIndex(dataframe1['Date'])

In [6]:
dataframe1.to_sql("new_data", engine, if_exists='replace', index=False)

2023